# Amazon SageMaker RL상에서 stable baseline으로 Roboschool 시뮬레이션 훈련
---

## 소개

Roboschool은 로봇 시스템에 대한 RL 정책을 훈련시키는 데 일반적으로 사용되는 [오픈 소스](https://github.com/openai/roboschool/tree/master/roboschool) 물리 시뮬레이터입니다. Roboschool은 다양한 로봇 문제에 해당하는 [다양한](https://github.com/openai/roboschool/blob/master/roboschool/__init__.py) gym 환경들을 제공합니다. 그 중 하나는 **HalfCheetah** 입니다. 이것은 다리가 두 개인 로봇으로, 수직 면(vertical plane)으로 제한되어 있어 앞,뒤로만 움직일 수 있습니다.


이 노트북 예제에서는 **HalfCheetah**가 걷는 법을 배우며, [OpenAI Baselines](https://github.com/openai/baselines) 를 기반으로 한 개선된 RL (Reinforcement Learning) 알고리즘의 구현체인 [stable-baselines](https://stable-baselines.readthedocs.io/en/master/) 를 사용합니다.

In [2]:
roboschool_problem = 'half-cheetah'

## 전제 조건(Pre-requisites)

### 라이브러리 임포트

시작하기 위해, 필요한 Python 라이브러리를 가져와서 권한 및 구성을 위한 몇 가지 전제 조건으로 환경을 설정합니다.

In [3]:
import sagemaker
import boto3
import sys
import os
import subprocess
from IPython.display import HTML
import time
from time import gmtime, strftime
sys.path.append("common")
from misc import get_execution_role, wait_for_s3_object
from docker_utils import build_and_push_docker_image
from sagemaker.rl import RLEstimator

### S3 버킷 설정

체크포인트(checkpoint) 및 메타데이터에 사용하려는 S3 버킷에 대한 연결 및 인증을 설정합니다.

In [4]:
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

S3 bucket path: s3://sagemaker-us-west-2-143656149352/


### 변수 설정

훈련 작업의 작업 접두사(job prefix)와 *컨테이너의 이미지 경로(BYOC 인 경우에만)와 같은 변수*를 정의합니다.

In [5]:
# create a descriptive job name 
job_name_prefix = 'rl-roboschool-'+roboschool_problem

### 훈련이 진행되는 위치 구성

SageMaker 노트북 인스턴스 또는 로컬 노트북 인스턴스를 사용하여 RL 훈련 작업을 훈련할 수 있습니다. 로컬 모드는 SageMaker Python SDK를 사용하여 SageMaker에 배포하기 전에 로컬 컨테이너에서 코드를 실행합니다. 이렇게 하면, 익숙한 Python SDK 인터페이스를 사용하면서 반복 테스트 및 디버깅 속도를 높일 수 있습니다. 여러분은 `local_mode = True` 만 설정하면 됩니다.

In [6]:
# run in local_mode on this machine, or as a SageMaker TrainingJob?
local_mode = False

if local_mode:
    instance_type = 'local'
else:
    # If on SageMaker, pick the instance type
    instance_type = "ml.c4.xlarge"

### IAM 역할 생성

SageMaker 노트북 `role = sagemaker.get_execution_role()`을 실행할 때 실행 역할(execution role)을 얻거나 로컬 시스템에서 실행할 때 utils 메소드 `role = get_execution_role()`을 사용하여 실행 역할을 작성하세요.

In [7]:
try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()
print("Using IAM role arn: {}".format(role))

Using IAM role arn: arn:aws:iam::143656149352:role/service-role/AmazonSageMaker-ExecutionRole-20200323T143836


### `로컬` 모드용 도커 설치

로컬 모드에서 작업하려면 도커(docker)가 설치되어 있어야 합니다. 로컬 머신에서 실행할 때는 docker 또는 docker-compose(로컬 CPU 머신의 경우) 및 nvidia-docker(로컬 GPU 머신의 경우)가 설치되어 있는지 확인하세요. 또는, SageMaker 노트북 인스턴스에서 실행할 때 다음 스크립트를 실행하여 관련 패키지들을 설치할 수 있습니다.

참고로, 한 번에 하나의 로컬 노트북만 실행할 수 있습니다.

In [8]:
# only run from SageMaker notebook instance
if local_mode:
    !/bin/bash ./common/setup.sh

## 도커 컨테이너 빌드

Roboschool이 설치된 사용자 정의 도커 컨테이너를 빌드해야 합니다. 컨테이너 빌드 작업은 아래 과정을 거쳐 처리됩니다.

1. 기본 컨테이너 이미지 가져오기
2. Roboschool 및 의존성 패키지 설치
3. stable-baselines 및 OpenMPI와 같은 의존성 패키지 설치
3. 새 컨테이너 이미지를 ECR에 업로드

인터넷 연결이 느린 컴퓨터에서 실행 중인 경우, 이 단계에서 시간이 오래 걸릴 수 있습니다. 노트북 인스턴스가 SageMaker 또는 EC2 인 경우 인스턴스 유형에 따라 3-10 분이 걸립니다.

In [9]:
%%time

cpu_or_gpu = 'gpu' if instance_type.startswith('ml.p') else 'cpu'
repository_short_name = "sagemaker-roboschool-stablebaselines-%s" % cpu_or_gpu
docker_build_args = { 
    'AWS_REGION': boto3.Session().region_name,
}
custom_image_name = build_and_push_docker_image(repository_short_name, build_args=docker_build_args)
print("Using ECR image %s" % custom_image_name)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Logged into ECR
Building docker image sagemaker-roboschool-stablebaselines-cpu from Dockerfile
$ docker build -t sagemaker-roboschool-stablebaselines-cpu -f Dockerfile . --build-arg AWS_REGION=us-west-2
Sending build context to Docker daemon  11.96MB
Step 1/41 : ARG AWS_REGION
Step 2/41 : FROM 520713654638.dkr.ecr.${AWS_REGION}.amazonaws.com/sagemaker-rl-tensorflow:coach0.11.0-cpu-py3
coach0.11.0-cpu-py3: Pulling from sagemaker-rl-tensorflow
7b8b6451c85f: Pulling fs layer
ab4d1096d9ba: Pulling fs layer
e6797d1788ac: Pulling fs layer
e25c5c290bde: Pulling fs layer
cd105da4078d: Pulling fs layer
f2c8922a63b9: Pulling fs layer
58eea52eed06: Pulling fs layer
adcd0d06f606: Pulling 

Removing intermediate container 385c4e927fd0
 ---> 1b41ddd3db12
Step 5/41 : RUN update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.9 50
 ---> Running in 9fd994b020b3
update-alternatives: using /usr/bin/gcc-4.9 to provide /usr/bin/gcc (gcc) in auto mode
Removing intermediate container 9fd994b020b3
 ---> 4a86b1ffb875
Step 6/41 : RUN apt-get install -y g++-4.9
 ---> Running in 889a84ccd6a3
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libstdc++-4.9-dev
Suggested packages:
  g++-4.9-multilib gcc-4.9-doc libstdc++6-4.9-dbg libstdc++-4.9-doc
The following NEW packages will be installed:
  g++-4.9 libstdc++-4.9-dev
0 upgraded, 2 newly installed, 0 to remove and 123 not upgraded.
Need to get 22.1 MB of archives.
After this operation, 45.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu xenial/universe amd64 libstdc++-4.9-dev amd64 4.9.3-13ubuntu2 [1126 kB]
Get:

  2700K .......... .......... .......... .......... .......... 17% 5.51M 2s
  2750K .......... .......... .......... .......... .......... 17% 8.47M 2s
  2800K .......... .......... .......... .......... .......... 17% 16.6M 2s
  2850K .......... .......... .......... .......... .......... 18% 7.99M 2s
  2900K .......... .......... .......... .......... .......... 18% 6.07M 2s
  2950K .......... .......... .......... .......... .......... 18% 28.6M 2s
  3000K .......... .......... .......... .......... .......... 19% 4.22M 2s
  3050K .......... .......... .......... .......... .......... 19% 12.1M 2s
  3100K .......... .......... .......... .......... .......... 19% 21.8M 2s
  3150K .......... .......... .......... .......... .......... 20% 6.11M 2s
  3200K .......... .......... .......... .......... .......... 20% 27.9M 2s
  3250K .......... .......... .......... .......... .......... 20% 8.10M 2s
  3300K .......... .......... .......... .......... .......... 20% 5.00M 2s
  3350K ....

  6700K .......... .......... .......... .......... .......... 42% 71.8M 1s
  6750K .......... .......... .......... .......... .......... 42% 7.31M 1s
  6800K .......... .......... .......... .......... .......... 42% 71.8M 1s
  6850K .......... .......... .......... .......... .......... 43% 82.0M 1s
  6900K .......... .......... .......... .......... .......... 43% 6.17M 1s
  6950K .......... .......... .......... .......... .......... 43% 9.44M 1s
  7000K .......... .......... .......... .......... .......... 44% 17.1M 1s
  7050K .......... .......... .......... .......... .......... 44% 60.7M 1s
  7100K .......... .......... .......... .......... .......... 44% 6.24M 1s
  7150K .......... .......... .......... .......... .......... 45% 32.8M 1s
  7200K .......... .......... .......... .......... .......... 45% 5.76M 1s
  7250K .......... .......... .......... .......... .......... 45% 70.0M 1s
  7300K .......... .......... .......... .......... .......... 46% 36.4M 1s
  7350K ....

 12200K .......... .......... .......... .......... .......... 76% 17.5M 0s
 12250K .......... .......... .......... .......... .......... 77% 98.9M 0s
 12300K .......... .......... .......... .......... .......... 77% 6.63M 0s
 12350K .......... .......... .......... .......... .......... 77% 56.5M 0s
 12400K .......... .......... .......... .......... .......... 78% 96.5M 0s
 12450K .......... .......... .......... .......... .......... 78% 5.89M 0s
 12500K .......... .......... .......... .......... .......... 78% 32.3M 0s
 12550K .......... .......... .......... .......... .......... 78%  113M 0s
 12600K .......... .......... .......... .......... .......... 79% 7.50M 0s
 12650K .......... .......... .......... .......... .......... 79% 38.1M 0s
 12700K .......... .......... .......... .......... .......... 79% 25.4M 0s
 12750K .......... .......... .......... .......... .......... 80% 7.63M 0s
 12800K .......... .......... .......... .......... .......... 80% 95.6M 0s
 12850K ....

...

Get:14 http://archive.ubuntu.com/ubuntu xenial/multiverse amd64 Packages [176 kB]
Get:15 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 Packages [1444 kB]
Get:16 http://archive.ubuntu.com/ubuntu xenial-updates/restricted amd64 Packages [13.1 kB]
Get:17 http://archive.ubuntu.com/ubuntu xenial-updates/universe amd64 Packages [1024 kB]
Get:18 http://archive.ubuntu.com/ubuntu xenial-updates/multiverse amd64 Packages [19.3 kB]
Get:19 http://archive.ubuntu.com/ubuntu xenial-backports/main amd64 Packages [7942 B]
Get:20 http://archive.ubuntu.com/ubuntu xenial-backports/universe amd64 Packages [8807 B]
Fetched 16.4 MB in 3s (4317 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libboost-python1.58-dev libboost-python1.58.0 libboost1.58-dev libpython-dev
  libpython-stdlib libpython2.7 libpython2.7-dev libpython2.7-minimal
  libpython2.7-stdlib python python-dev pyt

Removing intermediate container 46e02b019d6e
 ---> de9bdc9820ee
Step 31/41 : RUN apt-get update     && apt-get install -y --no-install-recommends python3.6-dev     && ln -s -f /usr/bin/python3.6 /usr/bin/python     && apt-get clean     && rm -rf /var/cache/apt/archives/* /var/lib/apt/lists/*
 ---> Running in ab68bd1065ff
Hit:1 http://security.ubuntu.com/ubuntu xenial-security InRelease
Hit:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu xenial InRelease
Hit:3 http://archive.ubuntu.com/ubuntu xenial InRelease
Hit:4 http://archive.ubuntu.com/ubuntu xenial-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu xenial-backports InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libpython3.6 libpython3.6-dev libpython3.6-minimal libpython3.6-stdlib
  python3.6 python3.6-minimal
Suggested packages:
  python3.6-venv python3.6-doc binfmt-support
The following packages wil

Get:14 http://archive.ubuntu.com/ubuntu xenial/multiverse amd64 Packages [176 kB]
Get:15 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 Packages [1444 kB]
Get:16 http://archive.ubuntu.com/ubuntu xenial-updates/restricted amd64 Packages [13.1 kB]
Get:17 http://archive.ubuntu.com/ubuntu xenial-updates/universe amd64 Packages [1024 kB]
Get:18 http://archive.ubuntu.com/ubuntu xenial-updates/multiverse amd64 Packages [19.3 kB]
Get:19 http://archive.ubuntu.com/ubuntu xenial-backports/main amd64 Packages [7942 B]
Get:20 http://archive.ubuntu.com/ubuntu xenial-backports/universe amd64 Packages [8807 B]
Fetched 16.4 MB in 3s (4341 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
cmake is already the newest version (3.5.1-1ubuntu3).
wget is already the newest version (1.17.1-1ubuntu1.5).
zlib1g-dev is already the newest version (1:1.2.8.dfsg-2ubuntu4.3).
zlib1g-dev set to manually installed.
The following additional pack

Selecting previously unselected package libhwloc-dev:amd64.
Preparing to unpack .../libhwloc-dev_1.11.2-3_amd64.deb ...
Unpacking libhwloc-dev:amd64 (1.11.2-3) ...
Selecting previously unselected package ocl-icd-libopencl1:amd64.
Preparing to unpack .../ocl-icd-libopencl1_2.2.8-1_amd64.deb ...
Unpacking ocl-icd-libopencl1:amd64 (2.2.8-1) ...
Selecting previously unselected package libhwloc-plugins.
Preparing to unpack .../libhwloc-plugins_1.11.2-3_amd64.deb ...
Unpacking libhwloc-plugins (1.11.2-3) ...
Selecting previously unselected package libibverbs-dev.
Preparing to unpack .../libibverbs-dev_1.1.8-1.1ubuntu2_amd64.deb ...
Unpacking libibverbs-dev (1.1.8-1.1ubuntu2) ...
Selecting previously unselected package libopenmpi-dev.
Preparing to unpack .../libopenmpi-dev_1.10.2-8ubuntu1_amd64.deb ...
Unpacking libopenmpi-dev (1.10.2-8ubuntu1) ...
Processing triggers for mime-support (3.59ubuntu1) ...
Processing triggers for libc-bin (2.23-0ubuntu10) ...
Setting up libpython3.5-minimal:amd64

Removing intermediate container 47101a8df801
 ---> 17b3c5f3e2a9
Successfully built 17b3c5f3e2a9
Successfully tagged sagemaker-roboschool-stablebaselines-cpu:latest
Done building docker image sagemaker-roboschool-stablebaselines-cpu
ECR repository already exists: sagemaker-roboschool-stablebaselines-cpu
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Logged into ECR
$ docker tag sagemaker-roboschool-stablebaselines-cpu 143656149352.dkr.ecr.us-west-2.amazonaws.com/sagemaker-roboschool-stablebaselines-cpu
Pushing docker image to ECR repository 143656149352.dkr.ecr.us-west-2.amazonaws.com/sagemaker-roboschool-stablebaselines-cpu

$ docker push 143656149352.dkr.ecr.us-west-2.amazonaws.com/sagemaker-roboschool-stablebaselines-cpu

## 훈련 코드 작성

#### RL 알고리즘에 대한 사전 설정 구성

RL 훈련 작업을 구성하는 사전 설정(preset)은 `./src` 디렉토리의 `preset-half-cheetah.py`에 정의되어 있습니다. 사전 설정 파일을 사용하여 에이전트 파라메터들을 정의하여 특정 에이전트 알고리즘을 선택할 수 있습니다. 환경 파라메터를 설정하고, 스케줄 및 시각화 파라메터를 정의하고, 그래프 관리자를 정의 할 수도 있습니다. 스케줄 사전 설정은 PPO1 알고리즘 훈련을 위해 다음과 같은 하이퍼파라미터들을 정의합니다.

* `num_timesteps`: (int) 훈련 스텝 수 - 사전 설정: 1e4
* `timesteps_per_actorbatch` – (int) 업데이트 당 액터 당 타입스텝 - 사전 설정: 2048
* `clip_param` – (float) clipping 파라메터 epsilon - 사전 설정: 0.2
* `entcoeff` – (float) entropy loss weight - 사전 설정: 0.0
* `optim_epochs` – (float) 옵티마이저의 에포크(epoch) 수 - 사전 설정: 10
* `optim_stepsize` – (float) 옵티마이저의 스텝 크기 - 사전 설정: 3e-4
* `optim_batchsize` – (int) 옵티마이저의 배치 크기 - 사전 설정: 64
* `gamma` – (float) 감가율(discount factor) - 사전 설정: 0.99
* `lam` – (float) advantage 추정 - 사전 설정: 0.95
* `schedule` – (str) 학습률 업데이트를 위한 스케줄러 유형 (‘linear’, ‘constant’, ‘double_linear_con’, ‘middle_drop’ 또는 ‘double_middle_drop’) - 사전 설정: linear
* `verbose` – (int) verbosity 수준: 0 없음, 1 훈련 정보, 2 tensorflow 디버그 - 사전 설정: 1

여러분은 https://stable-baselines.readthedocs.io/en/master/modules/ppo1.html 에서 PPO1 알고리즘에 대한 전체 인수 및 문서 목록을 참조할 수 있습니다.

`RLSTABLEBASELINES_PRESET` 하이퍼파라미터를 지정하여 런타임 시, 이를 오버라이드할 수 있습니다. 또한 사용자 정의 하이퍼파라미터를 정의하는 데 사용할 수 있습니다.

In [10]:
!pygmentize src/preset-{roboschool_problem}.py

import argparse

from sagemaker_rl.stable_baselines_launcher import SagemakerStableBaselinesPPO1Launcher, create_env


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--output_path', default="/opt/ml/output/intermediate/", type=str)
    parser.add_argument('--num_timesteps', default=1e4)
    parser.add_argument('--timesteps_per_actorbatch', default=2048, type=int)
    parser.add_argument('--clip_param', default=0.2, type=float)
    parser.add_argument('--entcoeff', default=0.0, type=float)
    parser.add_argument('--optim_epochs', default=10, type=int)
    parser.add_argument('--optim_stepsize', default=3e-4)
    parser.add_argument('--optim_batchsize', default=64, type=int)
    parser.add_argument('--gamma', default=0.99, type=float)
    parser.add_argument('--lam', default=0.95, type=float)
    parser.add_argument('--schedule', default="linear", type=str)
    parser.add_argument('--verbose', default=1, type=int)

    return parser.parse_known_args()


#### Write the Training Code

The training code is in the file `train-coach.py` which is also the `./src` directory.

In [11]:
!pygmentize src/train_stable_baselines.py

import argparse

from sagemaker_rl.mpi_launcher import MPILauncher


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--RLSTABLEBASELINES_PRESET', required=True, type=str)
    parser.add_argument('--output_path', default="/opt/ml/output/intermediate/", type=str)
    parser.add_argument('--instance_type', type=str)

    return parser.parse_known_args()


if __name__ == "__main__":
    args, unknown_args = parse_args()
    print("Launching train script with MPI: {} and arguments: {}".format(args.RLSTABLEBASELINES_PRESET,
                                                                         str(unknown_args)))
    MPILauncher(train_script=args.RLSTABLEBASELINES_PRESET, train_script_args=unknown_args,
                instance_type=args.instance_type).mpi_run()


## Python SDK 스크립트 모드를 사용하여 RL 모델 훈련

로컬 모드를 사용하는 경우 노트북 인스턴스에서 훈련이 실행됩니다. 훈련을 위해 SageMaker를 사용할 때 GPU 또는 CPU 인스턴스를 선택할 수 있습니다. RLEstimator는 RL 작업 훈련에 사용됩니다.

1. 환경, 사전 설정 및 교육 코드가 업로드되는 소스 디렉토리를 지정합니다.
2. 엔트리 포인트(entry point)를 훈련 코드로 지정합니다.
3. RL 툴킷 및 프레임 워크를 선택합니다. 이는 RL 컨테이너의 ECR 경로로 자동 확인됩니다.
4. 인스턴스 수, 작업 이름(job name), 출력에 대한 S3 경로 및 작업 이름과 같은 훈련 파라메터들을 정의합니다.
5. RL 에이전트 알고리즘의 하이퍼파라미터를 지정합니다. `RLSTABLEBASELINES_PRESET`을 사용하여 사용하려는 RL 에이전트 알고리즘을 지정할 수 있습니다.
6. 여러분의 로그에서 캡처할 지표(metric)를 정의합니다. CloudWatch 및 SageMaker Notebook에서도 시각화할 수 있습니다.

`preset-half-cheetah.py`에 구성된 모든 사전 설정 파라메터는 `hyperparameters` 블록에 재정의 값을 지정하여 재정의 할 수 있습니다.

**참고** : MPI 기반 작업의 경우 로컬 모드는 단일 인스턴스 작업에서만 지원됩니다. 로컬 모드를 사용하는 경우 `instance_type`을 `1`로 사용하세요.

In [19]:
%%time

estimator = RLEstimator(entry_point="train_stable_baselines.py",
                        source_dir='src',
                        dependencies=["common/sagemaker_rl"],
                        image_name=custom_image_name,
                        role=role,
                        train_instance_type=instance_type,
                        train_instance_count=10,
                        output_path=s3_output_path,
                        base_job_name=job_name_prefix,
                        hyperparameters={
                            "RLSTABLEBASELINES_PRESET":"preset-{}.py".format(roboschool_problem),
                            "num_timesteps":1e5,
                            "instance_type":instance_type
                        },
                        metric_definitions= [
                            {
                                "Name":"EpisodesLengthMean",
                                "Regex":"\[.*,.*\]\<stdout\>\:\| *EpLenMean *\| *([-+]?[0-9]*\.?[0-9]*) *\|"
                            },
                            {
                                "Name":"EpisodesRewardMean",
                                "Regex":"\[.*,.*\]\<stdout\>\:\| *EpRewMean *\| *([-+]?[0-9]*\.?[0-9]*) *\|"
                            },
                            {
                                "Name":"EpisodesSoFar",
                                "Regex":"\[.*,.*\]\<stdout\>\:\| *EpisodesSoFar *\| *([-+]?[0-9]*\.?[0-9]*) *\|"
                            }
                        ]
                    )

estimator.fit(wait=True)

2020-03-28 14:48:39 Starting - Starting the training job...
2020-03-28 14:48:58 Starting - Launching requested ML instances......
2020-03-28 14:50:05 Starting - Preparing the instances for training.........
2020-03-28 14:51:28 Downloading - Downloading input data
2020-03-28 14:51:28 Training - Downloading the training image.........
2020-03-28 14:53:03 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-03-28 14:52:58,055 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-03-28 14:52:58,058 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-03-28 14:52:58,193 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-03-28 14:52:58,206 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {
        "sagemake

bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-03-28 14:53:04,489 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-03-28 14:53:04,493 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-03-28 14:53:04,624 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-03-28 14:53:04,638 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {
        "sagemaker_estimator": "RLEstimator"
    },
    "channel_input_dirs": {},
    "current_host": "algo-2",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1",
        "algo-2",
        "algo-3",
        "algo-4",
        "algo-5",
        "algo-6",
        "algo-7",
        "algo-8",
        "algo-9",
        "algo-10"
    ],
    "hyperparameters": {
        "num_timesteps": 100

[1,3]<stdout>:Starting training for half cheetah with PPO1
[1,2]<stdout>:Starting training for half cheetah with PPO1
[1,3]<stdout>:Launching training script with stable baselines PPO1
[1,2]<stdout>:Launching training script with stable baselines PPO1
[1,0]<stdout>:Starting training for half cheetah with PPO1
[1,4]<stdout>:Starting training for half cheetah with PPO1
[1,7]<stdout>:Starting training for half cheetah with PPO1
[1,0]<stdout>:Launching training script with stable baselines PPO1
[1,5]<stdout>:Starting training for half cheetah with PPO1
[1,7]<stdout>:Launching training script with stable baselines PPO1
[1,9]<stdout>:Starting training for half cheetah with PPO1
[1,4]<stdout>:Launching training script with stable baselines PPO1
[1,5]<stdout>:Launching training script with stable baselines PPO1
[1,9]<stdout>:Launching training script with stable baselines PPO1
[1,1]<stdout>:Starting training for half cheetah with PPO1
[1,1]<stdout>:Launching training script with stable baselin

[1,5]<stdout>:Optimizing...
[1,5]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,0]<stdout>:Optimizing...
[1,0]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,7]<stdout>:Optimizing...
[1,7]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,2]<stdout>:Optimizing...
[1,2]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,6]<stdout>:Optimizing...
[1,6]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,8]<stdout>:Optimizing...
[1,8]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,3]<stdout>:Optimizing...
[1,3]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,4]<stdout>:Optimizing...
[1,4]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,1]<stdout>:Optimizing...
[1,1]<stdout

[1,5]<stdout>:Optimizing...
[1,5]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,0]<stdout>:Optimizing...
[1,0]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,6]<stdout>:Optimizing...
[1,6]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,2]<stdout>:Optimizing...
[1,2]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,3]<stdout>:Optimizing...
[1,3]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,7]<stdout>:Optimizing...
[1,7]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,8]<stdout>:Optimizing...
[1,8]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,1]<stdout>:Optimizing...
[1,1]<stdout>:     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
[1,4]<stdout>:Optimizing...
[1,4]<stdout


2020-03-28 14:54:42 Uploading - Uploading generated training model2020-03-28 14:54:37,969 sagemaker-containers INFO     Reporting training SUCCESS
Training process started by MPI on worker node %s stopped algo-10
2020-03-28 14:54:39,549 sagemaker-containers INFO     Reporting training SUCCESS
Training process started by MPI on worker node %s stopped algo-8
2020-03-28 14:54:40,130 sagemaker-containers INFO     Reporting training SUCCESS
Training process started by MPI on worker node %s stopped algo-7
2020-03-28 14:54:40,122 sagemaker-containers INFO     Reporting training SUCCESS
Training process started by MPI on worker node %s stopped algo-9
2020-03-28 14:54:39,834 sagemaker-containers INFO     Reporting training SUCCESS
Training process started by MPI on worker node %s stopped algo-2
2020-03-28 14:54:40,242 sagemaker-containers INFO     Reporting training SUCCESS
Training process started by MPI on worker node %s stopped algo-3
2020-03-28 14:54:40,374 sagemaker-containers INFO     Re

## 시각화

RL 훈련에는 시간이 오래 걸릴 수 있습니다. 따라서 훈련 작업이 동작하는 동안 훈련 작업의 진행 상황을 추적할 수 있는 다양한 방법들이 있습니다. 훈련 도중 일부 중간 출력이 S3에 저장되므로, 이를 캡처하도록 설정합니다.

### 훈련 롤아웃 비디오 가져오기

특정 롤아웃의 비디오는 훈련 중 S3에 기록됩니다. 여기서 우리는 사용 가능한 모든 비디오 클립을 가져오고 마지막 비디오를 렌더링합니다.

In [12]:
job_name = estimator.latest_training_job.job_name
print("Training job: %s" % job_name)

s3_url = "s3://{}/{}".format(s3_bucket,job_name)

if local_mode:
    output_tar_key = "{}/output.tar.gz".format(job_name)
else:
    output_tar_key = "{}/output/output.tar.gz".format(job_name)

intermediate_folder_key = "{}/output/intermediate".format(job_name)
output_url = "s3://{}/{}".format(s3_bucket, output_tar_key)
intermediate_url = "s3://{}/{}".format(s3_bucket, intermediate_folder_key)

print("S3 job path: {}".format(s3_url))
print("Output.tar.gz location: {}".format(output_url))
print("Intermediate folder path: {}".format(intermediate_url))
    
tmp_dir = "/tmp/{}".format(job_name)
os.system("mkdir {}".format(tmp_dir))
print("Create local folder {}".format(tmp_dir))
wait_for_s3_object(s3_bucket, intermediate_folder_key, tmp_dir) 

NameError: name 'estimator' is not defined

### RL 출력 비디오

In [1]:
import io
import base64
video = io.open("{}/rl_out.mp4".format(tmp_dir), 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

NameError: name 'tmp_dir' is not defined

### 학습된  HalfCheetah의 걷기 예시

다음과 같은 추가 구성으로 훈련 작업을 수행 후, 아래 비디오 클립과 같은 결과를 확인할 수 있습니다.
* `train_instance_count`: 10
* `train_instance_type`: ml.c4.xlarge
* `num_timesteps`: 1e7

위의 설정으로 모델을 훈련하는 데 약 40분이 소요됩니다. 저렴한 인스턴스와 더 많은 훈련 기간으로 유사한 결과를 얻을 수 있습니다.

In [18]:
import io
import base64
video = io.open("examples/robo_half_cheetah_10x_40min.mp4", 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))